In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
basedir = './prepped_data/'

In [3]:
%%time
main_data_df = pd.read_csv(basedir + 'main_data_baseline_checkpoint_2.csv')
main_data_df.shape

Wall time: 5.74 s


(4441256, 17)

In [4]:
%%time
land_prices_df = pd.read_csv(basedir + 'land_prices_ver_2.csv')
land_prices_df.shape

Wall time: 606 ms


(1594565, 4)

In [5]:
%%time
land_plans_df = pd.read_csv(basedir + 'land_plans_ver_3.csv')
land_plans_df.shape

<string>:2: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,27

Wall time: 31.9 s


(1565190, 288)

In [8]:
land_plans_df.fillna('해당사항없음', inplace=True)

In [9]:
dfs_list = [main_data_df, land_prices_df, land_plans_df]
for df in dfs_list:
    print(df.shape)

(4441256, 17)
(1594565, 4)
(1565190, 288)


In [10]:
main_data_df.rename(columns={'전용면적(㎡)':'전용면적'}, inplace=True)

In [11]:
%%time
main_data_df['전용면적_mean'] = main_data_df[['지번주소', '건축년도', '전용면적']].groupby(['지번주소', '건축년도']).transform('mean')['전용면적']
main_data_df['전용면적_min'] = main_data_df[['지번주소', '건축년도', '전용면적']].groupby(['지번주소', '건축년도']).transform('min')['전용면적']
main_data_df['전용면적_max'] = main_data_df[['지번주소', '건축년도', '전용면적']].groupby(['지번주소', '건축년도']).transform('max')['전용면적']

Wall time: 1.68 s


In [12]:
main_data_df = main_data_df[(main_data_df['전월세매매구분'] == '매매')|(main_data_df['전월세매매구분'] == '전세')].drop(columns=['월세(만원)', '도로명주소'])
print(main_data_df.shape)

(3607377, 18)


In [13]:
main_data_df['건물나이'] = main_data_df['년'] - main_data_df['건축년도']
main_data_df['건물나이'].describe()

count    3.607377e+06
mean     1.393074e+01
std      9.846349e+00
min     -1.200000e+01
25%      6.000000e+00
50%      1.300000e+01
75%      2.100000e+01
max      8.900000e+01
Name: 건물나이, dtype: float64

In [14]:
main_data_df['건물나이^2'] = main_data_df['건물나이']**2

In [15]:
main_data_df.drop(columns=['건축년도'], inplace=True)

In [16]:
main_data_df['year_linear'] = (main_data_df['년'] - main_data_df['년'].min()) + ((main_data_df['월']-1)/12)

In [17]:
def month_circular_sine_func(x):
    return math.sin(2*math.pi*((x-1)/12))

def month_circular_cosine_func(x):
    return math.cos(2*math.pi*((x-1)/12))

In [18]:
main_data_df['month_sin'] = main_data_df['월'].apply(month_circular_sine_func)
main_data_df['month_cos'] = main_data_df['월'].apply(month_circular_cosine_func)

In [19]:
main_data_df.drop(columns=['월', '계약일'], inplace=True)

In [20]:
xmax = main_data_df['X좌표'].max()
xmin = main_data_df['X좌표'].min()
ymax = main_data_df['Y좌표'].max()
ymin = main_data_df['Y좌표'].min()

In [21]:
coor_minmax_df = pd.DataFrame({'xmax':xmax, 'xmin':xmin, 'ymax':ymax, 'ymin':ymin}, index=[0])
coor_minmax_df.to_csv('./Training/training_data_ver_10/coor_minmax.csv', index=False)
coor_minmax_df

,xmax,xmin,ymax,ymin
0,971728.508488,938089.149737,1.965830e+06,1.937374e+06


In [22]:
main_data_df['xnorm'] = (main_data_df['X좌표'] - xmin - ((xmax - xmin)/2)) / 10000
main_data_df['ynorm'] = (main_data_df['Y좌표'] - ymin - ((ymax - ymin)/2)) / 10000
main_data_df['x_2nd'] = main_data_df['xnorm']**2 / 2
main_data_df['y_2nd'] = main_data_df['ynorm']**2 / 2
main_data_df['x_sin'] = main_data_df['xnorm'].apply(np.sin)
main_data_df['y_sin'] = main_data_df['ynorm'].apply(np.sin)
main_data_df['x_cos'] = main_data_df['xnorm'].apply(np.cos)
main_data_df['y_cos'] = main_data_df['ynorm'].apply(np.cos)
main_data_df['xy'] = main_data_df['xnorm'] * main_data_df['ynorm']

In [23]:
main_data_df.drop(columns=['X좌표', 'Y좌표'], inplace=True)

In [24]:
main_data_df['trade_type'] = main_data_df['건물종류'] + '_' + main_data_df['전월세매매구분']
main_data_df.drop(columns=['건물종류', '전월세매매구분'], inplace=True)

In [27]:
merge1_df = main_data_df.merge(land_prices_df, on=['지번주소', '년'])
merge1_df.shape

(3493309, 28)

In [28]:
merge1_df['공시지가'] = merge1_df['공시지가'] / 10000
merge1_df.rename(columns={'공시지가':'공시지가(만원)'}, inplace=True)

In [46]:
merge3_df = merge1_df.merge(land_plans_df, on=['지번주소', '년'])
merge3_df.shape

(3490574, 314)

In [47]:
merge3_df.drop(columns=['지번주소'], inplace=True)

In [48]:
merge3_df.columns = [col.replace(':','').replace('/','').replace('\\','').replace('*','').replace('?','').replace('\"','')\
                     .replace('|','').replace('<','').replace('>','') for col in merge3_df.columns]

In [49]:
land_plans_df.columns = [col.replace(':','').replace('/','').replace('\\','').replace('*','').replace('?','').replace('\"','')\
                     .replace('|','').replace('<','').replace('>','') for col in land_plans_df.columns]

In [50]:
lp_catcols = land_plans_df.columns.tolist()[2:]
lp_catcols[:5]

['(한강)오염행위 제한지역', '(한강)폐기물매립시설 설치제한지역', '4대문안', '가로구역별 최고높이 제한지역', '가스공급설비']

In [51]:
merge3_df['전용면적'] = merge3_df['전용면적'].apply(np.log)

In [52]:
merge3_df.drop(columns=['년'], inplace=True)

In [53]:
merge3_df.head()

,전용면적,금액(만원),층,가격면적,target_log_transformed,targetarea_log_transformed,전용면적_mean,전용면적_min,전용면적_max,건물나이,건물나이^2,year_linear,month_sin,month_cos,xnorm,ynorm,x_2nd,y_2nd,x_sin,y_sin,x_cos,y_cos,xy,trade_type,공시지가(만원),표준지여부,(한강)오염행위 제한지역,(한강)폐기물매립시설 설치제한지역,4대문안,가로구역별 최고높이 제한지역,가스공급설비,가축사육제한구역,개발제한구역,개발진흥지구,개발행위허가제한지역,건축선,건축용도지역기타,건축용도지역미분류,건축허가·착공제한지역,경관광장,경관녹지,경관지구,경관지구기타,고도지구,고등학교,고속철도,공공공지,공공도서관,공공문화체육시설미분류,공공시설구역,공공주택지구,공공지원민간임대주택 공급촉진지구,공공청사,공동구,공용시설보호지구,공원,공원마을지구(공원집단시설지구),공원자연보존지구,공원자연환경지구,공익용산지,공익임지,공장설립승인지역,공장설립제한지역,공항소음피해예상지역,공항시설보호지구,과밀억제권역,광로1류(폭 70M 이상),광로2류(폭 50M~70M),광로3류(폭 40M~50M),광역계획구역,광장,교육환경보호구역,교통광장,국가산업단지,국가지정문화재구역,국가하천,국립공원,국민임대주택단지예정지구,국지도로,국토이용기타용도지구,국토이용용도지구기타,근린공원,근린광장,근린상업지역,기타공공공지시설,기타공공청사시설,기타공원시설,기타교통시설,기타녹지시설,기타도로시설,기타도시공간시설,기타문화시설,기타보건위생시설,기타수도시설,기타시장시설,기타용도지역지구기타,기타용지,기타유통및공급시설,기타유통업무설비,기타자동차정류장,기타주차장시설,기타철도시설,기타폐기물처리시설,노외주차장,녹지,대공방어협조구역,대공방어협조구역(위탁고도54-236m),대공방어협조구역(위탁고도77-257m),대로1류(폭 35M~40M),대로2류(폭 30M~35M),대로3류(폭 25M~30M),대학,도로,도로구역,도서관,도시개발구역,도시개발구역기타,도시고속도로,도시관리계획 입안중,도시기타용도지역지구기타,도시기타용도지역지구미분류,도시기타용도지역지구용도지역지구,도시자연공원,도시자연공원구역,도시지역,도시지역기타,도시철도,묘지공원,문화공원,문화시설,문화재,문화재보존영향 검토대상구역,문화재보호구역,문화재보호구역기타,문화지구,박물관,방송통신시설,방수설비,방재지구,방화지구,변전소(전원개발사업구역),변전시설,보전산지,보전임지,보존지구,보행자전용도로,부설주차장,비오톱1등급,비행안전제2구역(전술),비행안전제2구역(지원),비행안전제3구역(전술),비행안전제3구역(지원),비행안전제4구역(지원),비행안전제5구역(전술),비행안전제5구역(지원),비행안전제6구역(전술),사고지,사방시설,사방지,사업지역기타,사업지역미분류,사회복지시설,산업개발진흥지구,산업시설구역,상대보호구역,상대정화구역,상수원보호구역,상수원보호기타,생산녹지지역,소공원,소로1류(폭 10M~12M),소로2류(폭 8M~10M),소로3류(폭 8M 미만),소하천,소하천구역,수도공급시설,수평표면구역,시·도 생태·경관보전지역,시가지경관지구,시가지조성사업지역,시도지정문화재구역,시장,시장정비구역,시장정비구역기타,아파트지구,야생생물보호구역,어린이공원,여객자동차터미널,역사도심,역사문화미관지구,역사문화특화경관지구,역사문화환경보존지역,연결녹지,연구개발특구,연구시설,열공급설비,온천공보호구역,완충녹지,용도구역기타,용도구역미분류,운동장,원추표면구역,원형보존지,유수지,유원지,유치원,유통단지,유통상업지역,유통업무설비,일단의공업용지조성사업지역,일단의주택단지조성사업지역,일반도로,일반미관지구,일반산업단지,일반상업지역,일반주거지역,일반철도,임업용산지,자동차검사시설,자동차전용도로,자동차정류장,자연경관지구,자연녹지지역,자연재해위험지구,장애물제한표면구역,재개발구역,재개발구역기타,재정비촉진지구,재정비촉진지구기타,저류시설,전기공급설비,전용주거지역,전이표면구역,전통사찰보존구역,절대보호구역,절대정화구역,정비구역,정비구역기타,정비예정구역,제1종일반주거지역,제1종전용주거지역,제1종지구단위계획구역,제2종일반주거지역,제2종전용주거지역,제3종 구역,제3종일반주거지역,제방,제한보호구역,제한보호구역(전술항공5km),제한보호구역(후방지역500m),조례로정한지역,조망가로미관지구,조망가로특화경관지구,종합의료시설,주간선도로,주거용지,주거환경개선지구,주차장,주차장기타,주차환경개선지구,준공업지역,준보전산지,준주거지역,중로1류(폭 20M~25M),중로2류(폭 15M~20M),중로3류(폭 12M~15M),중심대광장,중심상업지역,중심지미관지구,중요시설물보존지구,중요시설물보호지구,중요시설물보호지구(공용),중요시설물보호지구(공항),중점경관관리구역,중학교,지구단위계획구역,지역특화발전특구,지원시설구역,지정문화재구역,지하광장,지하도로,진입표면구역,집단취락지구,철도,청사,청소년수련시설,체육공원,체육시설,초등학교,최고고도지구,침수위험지구,택지개발예정지구,택지개발예정지구기타,토지거래계약에관한허가구역,토지구획정리사업지구기타,통제보호구역,통제보호구역(민통선이남300m),특수도로(보행자전용도로),특정개발진흥지구,특화경관지구,폐기물처리및재활용시설,폐기물처리시설,하수도,하수종말처리시설,하천,하천구역,학교,학교시설보호지구,학교이적지,학교환경위생 정화구역,현상변경허가 대상구역,홍수관리구역
0,4.353499,57000,2,733.118971,10.950807,6.597308,77.75,77.75,77.75,25,625,7.666667,-0.866025,-0.500000,0.556491,-0.904896,0.154841,0.409418,0.52821,-0.786361,0.849114,0.617768,-0.503567,아파트_매매,921.0,N,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,포함,해당사항없음,접함,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,포함,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,접함,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,포함,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,포함,포함,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,해당사항없음,

In [55]:
def floor_feature_engi(x):
    # x is a number
    
    if pd.isna(x) == True:
        return x:
    else:
    
        if x < 0:
            return '지하'
        elif (x == 0)|(x == 1):
            return '1층'
        elif x == 2:
            return '2층'
        elif x == 3:
            return '3층'
        elif x == 4:
            return '4층'
        elif x == 5:
            return '5층'
        elif x == 6:
            return '6층'
        elif x == 7:
            return '7층'
        elif x == 8:
            return '8층'
        elif x == 9:
            return '9층'
        elif x == 10:
            return '10층'
        elif x < 14:
            return '13층이하'
        elif x < 17:
            return '16층이하'
        elif x < 20:
            return '19층이하'
        elif x < 25:
            return '24층이하'
        elif x < 30:
            return '29층이하'
        elif x < 35:
            return '34층이하'
        elif x < 40:
            return '39층이하'
        elif x < 45:
            return '44층이하'
        elif x < 50:
            return '49층이하'
        else:
            return '50층이상'

In [56]:
merge3_df['층'] = merge3_df['층'].apply(floor_feature_engi)
merge3_df['층'].value_counts()

2층       461670
3층       437283
4층       376866
13층이하    351766
1층       286720
5층       282597
16층이하    233363
6층       183856
7층       160137
8층       147584
9층       141884
10층      135725
19층이하    114488
24층이하     86557
지하        54534
29층이하     23291
34층이하      7462
39층이하      2841
44층이하       854
49층이하       564
50층이상       532
Name: 층, dtype: int64

In [57]:
merge3_df.drop(columns=['건물나이^2'], inplace=True)

In [58]:
def building_age_feature_engi(x):
    # x is a number
    
    if pd.isna(x) == True:
        return x
    else:
        
        if x < 0:
            return '건축전계약'
        elif x < 6:
            return '5년이하'
        elif x < 11:
            return '10년이하'
        elif x < 16:
            return '15년이하'
        elif x < 20:
            return '19년이하'
        elif x < 30:
            return '29년이하'
        elif x < 40:
            return '39년이하'
        else:
            return '40년이상'

In [59]:
merge3_df['건물나이'] = merge3_df['건물나이'].apply(building_age_feature_engi)
merge3_df['건물나이'].value_counts()

5년이하     764528
29년이하    717017
15년이하    657039
10년이하    635306
19년이하    435396
39년이하    248569
40년이상     31239
건축전계약      1480
Name: 건물나이, dtype: int64

In [60]:
cat_cols = ['trade_type', '표준지여부', '층', '건물나이'] + lp_catcols

In [61]:
for col in tqdm(cat_cols):
    col_map = {}
    col_map_df = pd.DataFrame({'cat':merge3_df[col].unique(),'map':[i for i in range(len(merge3_df[col].unique()))]})
    for i in range(col_map_df.shape[0]):
        col_map[col_map_df['cat'][i]] = col_map_df['map'][i]
    merge3_df[col] = merge3_df[col].map(col_map)
    col_map_df.to_csv('./Training/training_data_ver_11/feature_maps/{}_map.csv'.format(col))

100%|████████████████████████████████████████████████████████████████████████████████| 290/290 [08:25<00:00,  1.74s/it]


In [62]:
cat_cols_df = pd.DataFrame({'colname':cat_cols})
cat_cols_df.head()

,colname
0,trade_type
1,표준지여부
2,층
3,건물나이
4,(한강)오염행위 제한지역


In [63]:
cat_cols_df.to_csv('./Training/training_data_ver_11/cat_cols.csv', index=False)

In [64]:
cols_to_drop = []
for col in tqdm(merge3_df.columns):
    try:
        if merge3_df[col].std() == 0:
            cols_to_drop.append(col)
        else:
            pass
    except:
        pass

100%|████████████████████████████████████████████████████████████████████████████████| 311/311 [00:09<00:00, 33.20it/s]


In [65]:
len(cols_to_drop)

1

In [66]:
cols_to_drop

['기타유통및공급시설']

In [67]:
merge3_df.drop(columns=cols_to_drop, inplace=True)

In [68]:
%%time
merge3_df.to_csv('./Training/training_data_ver_11/training_data_ver_11.csv', index=False)

Wall time: 12min 54s
